In [ ]:
import os
import numpy as np
import cv2 as cv
import json
from rich import print
from copy import deepcopy
from rich.console import Console
from scipy.stats import mode
import math
import skimage
# from skimage import morphology
import matplotlib.pyplot as plt
from plantcv import plantcv as pcv
from scipy import ndimage
import pandas as pd
console = Console()
save_path = 'final_results'

In [ ]:
def CreateDir(save_path, path):
    dir_path = os.path.join(save_path, path[:-4])
    
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)
    else:
        console.print('{} already exist!'.format(path[:-4]), style = 'green')
    return dir_path
        
def ChangePoints(points):
    '''
    将坐标点从左边移到右边
    '''
    right_points = []
    for point in points:
        new_point = point
        new_point[0] = point[0] + 550
        right_points.append(new_point)
    return right_points
def GetLableMask(image, data1, dir_path, save_path):
    '''
    从json文件中获得标签数据，并转化成图像。
    '''
    save_image = image.copy()
    img_height = data1['imageHeight']
    img_width = data1['imageWidth']
    label_mask = np.zeros((img_height, img_width), dtype=np.uint8)
    left_label_mask = label_mask.copy()
    right_label_mask = label_mask.copy()
    shape = data1['shapes'][0]
    # print(shape)
    fnab = 0 if shape['label'][0:6] == 'benign' else 1
    rod = shape['label'][6:] if fnab == 0 else shape['label'][9:]
    points = shape['points']   
    polygons = np.array(points, dtype=np.int32)
    cv.polylines(save_image, [polygons], isClosed=True, color=(0, 0, 255), thickness=2)
    cv.fillPoly(left_label_mask, [polygons], 255)
    # cv.polylines(left_label_mask, [polygons], True, 255, thickness=-1)
    if dir_path[-1] == '1':
        right_label_mask = left_label_mask.copy()
    else:
        points1 = ChangePoints(points)
        polygons = np.array(points1, dtype=np.int32)
        cv.polylines(save_image, [polygons], isClosed=True, color=(0, 0, 255), thickness=2)
        cv.fillPoly(right_label_mask, [polygons], 255)
    # cv.polylines(right_label_mask, [polygons], True, 255, thickness=-1)
    
    # nodule_mask = cv.bitwise_and(image, image, mask = right_label_mask)
    
    x_indices, y_indices = np.where(right_label_mask)
    x_min, x_max = np.min(x_indices), np.max(x_indices)
    y_min, y_max = np.min(y_indices), np.max(y_indices)
    # print(x_min, x_max, y_min, y_max)
    rectangle_roi = image[x_min:x_max, y_min:y_max, :]
    nodule_mask = right_label_mask[x_min:x_max, y_min:y_max]
    index = [x_min, x_max, y_min, y_max]
    
    cv.imwrite(save_path + '/' + '1.png', save_image)
    return fnab, left_label_mask, right_label_mask, nodule_mask, rectangle_roi, index, rod

# def Save1 (save_path, path, left_label_maask, right_label_mask, image):
#     dir_path = os.path.join(save_path, path[:-4])
#     image_name = dir_path + '/' + '1.png'
#     cv.imwrite(image_name, image)
#     original_gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
#     mask_gray = cv2.cvtColor(mask_image, cv2.COLOR_BGR2GRAY)

In [ ]:


def AutoSegVesselInNodule(roi_region):
    # masked_image = cv.bitwise_and(image, image, mask=mask_nodule)
    # hsv = cv.cvtColor(roi_region, cv.COLOR_BGR2HSV)
    # lower_blue = np.array([0, 0, 127])
    # upper_blue = np.array([255, 255, 255])
    # vessel_mask = cv.inRange(roi_region, lower_blue, upper_blue)
    # vessel_region = cv.bitwise_and(roi_region, roi_region, mask=vessel_mask)
    # cv.imshow('vessel_mask', vessel_region)
    # cv.waitKey(0)
    new_roi_region = cv.cvtColor(roi_region, cv.COLOR_BGR2HSV)
    # cv.imshow('vessel_mask', new_roi_region[0])
    # cv.waitKey(0)
    vessel_mask = np.zeros(new_roi_region.shape[0:2])
    for i in range(roi_region.shape[0]):
        for j in range(roi_region.shape[1]):
            if roi_region[i, j, 0] < 110:
                # print('123')
                vessel_mask[i,j] = 0
            else:
                vessel_mask[i,j] = 255
    return vessel_mask.astype(np.uint8)
# 定义鼠标回调函数
def on_mouse_click(event, x, y, flags, param):
    if event == cv.EVENT_LBUTTONDOWN:
        console.print("鼠标左键点击位置的像素值为:", vessel_image[y, x, :], style='red')

def CheckHole(vessel_mask, rectangle_roi):
    # bit_not = cv.bitwise_not(vessel_mask)
    # num_labels, labels, stats, centroids = cv.connectedComponentsWithStats(bit_not, connectivity=8, ltype=None)
    # plt.imshow(labels)
    contours, hierarchy = cv.findContours(vessel_mask, cv.RETR_CCOMP, cv.CHAIN_APPROX_SIMPLE)
    for i in range(len(contours)):
        cv.drawContours(vessel_mask, contours, i, 255, -1)
    return vessel_mask

def ImageProcess(rectangle_roi):
    gray_roi_image = cv.cvtColor(rectangle_roi, cv.COLOR_BGR2GRAY)
    frangi_image = skimage.filters.frangi(gray_roi_image, scale_range=(1, 20), scale_step=1, alpha=0.5, 
                                        beta=15, gamma=2, black_ridges=False)
    frangi_image_uint8 = (frangi_image - frangi_image.min()) / (frangi_image.max() - frangi_image.min()) * 255
    frangi_image_uint8 = frangi_image_uint8.astype(np.uint8)

    binary_frangi_image, otsu_thresh = cv.threshold(frangi_image_uint8, 0, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)
    cv.imshow('frangi', otsu_thresh)
    cv.waitKey(0)
    kernel = np.ones((3, 3), np.uint8)
    open_operation_image = cv.morphologyEx(otsu_thresh, cv.MORPH_TOPHAT, kernel)
    cv.imshow('tophat', open_operation_image)
    cv.waitKey(0)
    vessel_mask = otsu_thresh - open_operation_image
    
    vessel_image = cv.bitwise_and(rectangle_roi, rectangle_roi, mask=otsu_thresh)
    # vessel_image = cv.cvtColor(vessel_image, cv.COLOR_BGR2GRAY)
    vessel_mask = AutoSegVesselInNodule(vessel_image)
    
    vessel_mask = CheckHole(vessel_mask, rectangle_roi)
    # cv.imshow('otsu', vessel_mask)
    
    num_labels, labels, stats, centroids = cv.connectedComponentsWithStats(vessel_mask, connectivity=8)
    # plt.imshow(labels)
    # plt.show()
    # 去除面积小的连通域
    # labels = labels.astype(np.uint8)
    min_area = 10  # 设置最小面积阈值
    for i in range(1, num_labels):  # 跳过背景
        if np.count_nonzero(labels == i) < min_area:
            labels[labels == i] = 0
        else:
            labels[labels == i] = 100000
    labels[labels == 100000] = 255
    labels = labels.astype(np.uint8)
    # cv.imshow('otsu1', labels)
    # cv.waitKey(0)
    # # 定义结构元素
    # kernel = np.ones((3,3),np.uint8)

    # # 膨胀操作
    # dilated_image = cv.dilate(labels, kernel, iterations=1)

    # # 腐蚀操作
    # eroded_image = cv.erode(dilated_image, kernel, iterations=1)
    # 重新标记连通域
    # num_labels, labels = cv.connectedComponents(labels, connectivity=8)

    # # 可视化结果
    # output_image = np.zeros_like(vessel_mask)
    # for i in range(1, num_labels):
    #     output_image[labels == i] = 255

    # kernel = np.ones((5, 5), np.uint8)
    # vessel_mask = vessel_mask - cv.morphologyEx(vessel_mask, cv.MORPH_TOPHAT, kernel)
    vessel_image = cv.bitwise_and(rectangle_roi, rectangle_roi, mask=labels)
    
    return labels, vessel_image

def IsPowerOfTwo(n):
    return n > 0 and (n & (n - 1)) == 0
def Calnum_s(new_img, start_s):
    kernel = np.ones((start_s, start_s))
    img_height, img_width = new_img.shape
    kernel_height, kernel_width = kernel.shape
    output_height = (img_height - kernel_height) // start_s + 1
    output_width = (img_width - kernel_width) // start_s + 1
    output_img = np.zeros((output_height, output_width))
    for y in range(0, output_height):
        for x in range(0, output_width):
            output_img[y, x] = np.sum(new_img[y*start_s:y*start_s+kernel_height, x*start_s:x*start_s+kernel_width] * kernel) / 255
    # print(output_height * output_width - np.count_nonzero(output_img))  
    return np.count_nonzero(output_img)  
def CalmvFP(vessel_mask):
    new_size = max(vessel_mask.shape[0], vessel_mask.shape[1])
    while IsPowerOfTwo(new_size) != True:
        new_size += 1
    new_img = np.zeros((new_size, new_size), dtype=np.uint8)
    x_offset = (new_size - vessel_mask.shape[1]) // 2
    y_offset = (new_size - vessel_mask.shape[0]) // 2
    new_img[y_offset:y_offset+vessel_mask.shape[0], x_offset:x_offset+vessel_mask.shape[1]] = vessel_mask
    x_list = []
    y_list = []
    start_s = new_size
    while start_s >= 1:
        x_list.append(math.log2(1 / start_s))
        # print(start_s)
        # cv.imshow('new_img', new_img)
        # cv.waitKey(0)
        # print(Calnum_s(new_img, start_s))
        y_list.append(math.log2(Calnum_s(new_img, start_s)))
        start_s = int(start_s / 2)
    mvFP = 1000000 if np.isnan(np.polyfit(x_list, y_list, 1)[0]) else np.polyfit(x_list, y_list, 1)[0]
    # plt.scatter(x_list, y_list, label='Data')
    # plt.plot(x_list, np.polyfit(x_list, y_list, 1)[0] * np.array(x_list) + np.polyfit(x_list, y_list, 1)[1], color='red', label='Fitted Line')
    # plt.title('mvFP Fitted Line')
    # plt.xlabel('x')
    # plt.ylabel('y')
    # plt.legend()
    # plt.show()
    return mvFP
def FindTips(obj):
    tips = []
    # print(obj.shape)
    if obj.shape[0] < 3:
        # print(123)
        tips.append(obj[0][0])
        tips.append(obj[0][0])
    else:
        list_points = []
        for i in range(obj.shape[0]):
            list_points.append(obj[i][0])
        points = []
        for matrix in list_points:
            # 判断当前矩阵是否已经存在于unique_matrices中
            exists = any(np.array_equal(matrix, m) for m in points)
            # 如果不存在，则添加到unique_matrices中
            if not exists:
                points.append(matrix)
        for point in points:
            surround_points = []
            surround_points.append(np.array([point[0] - 1, point[1] - 1]))
            surround_points.append(np.array([point[0] - 1, point[1]]))
            surround_points.append(np.array([point[0] - 1, point[1] + 1]))
            surround_points.append(np.array([point[0], point[1] - 1]))
            surround_points.append(np.array([point[0], point[1] + 1]))
            surround_points.append(np.array([point[0] + 1, point[1] - 1]))
            surround_points.append(np.array([point[0] + 1, point[1]]))
            surround_points.append(np.array([point[0] + 1, point[1] + 1]))
            surround_num = 0
            for matrix in surround_points:
                exists = any(np.array_equal(matrix, m) for m in points)
                if exists:
                    surround_num += 1
            # intersection = list(points & surround_points)
            if surround_num == 1:
                tips.append(point)
    return tips
def ExtractSkeleton(vessel_mask):
    mask1 = vessel_mask.copy()
    mask1 = mask1 / 255
    skeleton = pcv.morphology.skeletonize(mask1).astype(np.uint8)
    mask1 = cv.cvtColor(vessel_mask, cv.COLOR_GRAY2BGR)
    mask1[skeleton == 1, :] = [0, 255, 0] 
    skeleton = skeleton * 255
    pcv.params.debug = "None"
    pruned_skeleton, segmented_img, segment_objects = pcv.morphology.prune(skel_img=skeleton, size=0)
    return pruned_skeleton, segmented_img, segment_objects
def FindingTheSutiableDiameter(dis, branch, vessel_mask, obj, num):
    dis_map = cv.bitwise_and(dis, dis, mask=branch)
    non_zero_values = dis_map[dis_map != 0].astype(int)
    return np.median(non_zero_values) * 2
def calculate_average(nums):
    total = sum(nums)
    average = total / len(nums)
    return average

In [ ]:
features = {
    'name': [],
    'nv': [],
    'nb': [],
    'vd': [],
    'vdr': [],
    'svp': [],
    'diameters': [],
    'ave_diameter': [],
    'max_diameter': [],
    'min_diameter': [],
    'md_list': [],
    'ave_md':[],
    'max_md': [],
    'min_md': [],
    'mvFP': [],
    'tortuosity_list': [],
    'ave_tortuosity': [],
    'max_tortuosity': [],
    'min_tortuosity': [],
    'ba_list': [],
    'ave_ba': [],
    'max_ba': [],
    'min_ba': [],
    'area': [],
    'noddule_dia': [],
    'class': []
}
dir_paths = ['all_data/benign', 'all_data/benign1', 'all_data/malignant', 'all_data/malignant1']
for dir_path in dir_paths:
    files = os.listdir(dir_path)
    for file in files:
        if file[-4:] != 'json':
            continue
        # if file != 'CHEN_YAN20231009-163028-6406CHEN_YAN20231009-163028-6406_0.json':
        #     continue
        image_path = file[:-4] + 'png'
        image = cv.imread(os.path.join(dir_path, image_path))
        try:
            if(image == None):
                image_path = file[:-4] + 'JPG'
                image = cv.imread(os.path.join(dir_path, image_path))
        except:
            pass
        print(file)
        json_path = os.path.join(dir_path, file)


    # image = cv.imread('data2-12.11/data2-12.11/data2/benign/CHEN_YAN20231009-163028-6406CHEN_YAN20231009-163028-6406_0.png')
    # json_path = 'data2-12.11/data2-12.11/data2/benign/CHEN_YAN20231009-163028-6406CHEN_YAN20231009-163028-6406_0.json'
        with open(json_path, 'r') as f:
            data = json.load(f)
        # label_data = data.copy()


            dir_path1 = CreateDir(save_path, file)
            data1 = deepcopy(data)
            fnab, left_label_mask, right_label_mask, nodule_mask, rectangle_roi, index, rod = GetLableMask(image, data1, dir_path, dir_path1)
            #血管区域提取
            vessel_mask, vessel_image = ImageProcess(rectangle_roi)
            vessel_mask = cv.bitwise_and(vessel_mask, vessel_mask, mask=nodule_mask)
            vessel_image = cv.bitwise_and(vessel_image, vessel_image, mask=nodule_mask)
            cv.imwrite(dir_path1 + '/' + '2.png', rectangle_roi)
            cv.imwrite(dir_path1 + '/' + '3.png', vessel_mask)
            cv.imwrite(dir_path1 + '/' + '4.png', vessel_image)
            # cv.imshow('mask', nodule_mask)
            # cv.waitKey(0)
            #血管骨架提取
            pruned_skeleton, segmented_img, segment_objects = ExtractSkeleton(vessel_mask)
            #查找并计算分支点数NB
            branch_points_list = pcv.morphology.find_branch_pts(pruned_skeleton)
            nonzero_coords = np.transpose(np.nonzero(branch_points_list))
            pruned_skeleton_show_image = cv.cvtColor(pruned_skeleton * 255, cv.COLOR_GRAY2BGR)
            for i in range(nonzero_coords.shape[0]):
                pruned_skeleton_show_image = cv.circle(pruned_skeleton_show_image, (nonzero_coords[i][1], nonzero_coords[i][0]), 1, (0, 0, 255), -1)
            # cv.imshow('prs', pruned_skeleton_show_image)
            # cv.waitKey(0)
            pruned_skeleton_show_image[branch_points_list == 255, :] = np.array((0, 0, 255))
            cv.imwrite(dir_path1 + '/' + '5.png', pruned_skeleton_show_image)
            nb = int(np.sum(branch_points_list) / 255)
            # console.print('number of branch points (NB):{}'.format(nb), style='yellow')
            #计算血管段数NV

            leaf_obj, other_obj = pcv.morphology.segment_sort(skel_img=pruned_skeleton,
                                                            objects=segment_objects)
            leaf_img = np.zeros_like(vessel_mask)
            for i, cnt in enumerate(leaf_obj):
                cv.drawContours(leaf_img, leaf_obj, i, 255, 1, lineType=8)
            for i, cnt in enumerate(other_obj):
                cv.drawContours(leaf_img, other_obj, i, 255, 1, lineType=8)
            nv = int(len(leaf_obj) + len(other_obj))
            # console.print('number of vessel branches (NV):{}'.format(nv), style='yellow')
            #计算血管密度VD
            vessel_area = np.sum(vessel_mask) / 255
            nodule_area = np.sum(right_label_mask) / 255
            vd = vessel_area / nodule_area
            # console.print('vessel density (VD):{}'.format(vd), style='yellow')
            #计算血管分布模式SVP和血管面积比VDR
            center_area = cv.resize(nodule_mask, None, fx=0.5, fy=0.5)
            center_x = int(nodule_mask.shape[0] / 2)
            center_y = int(nodule_mask.shape[1] / 2)
            center_area_image = np.zeros_like(nodule_mask)
            center_area_image[(center_x - int(center_area.shape[0] / 2)):(center_x - int(center_area.shape[0] / 2) + center_area.shape[0]),
                            (center_y - int(center_area.shape[1] / 2)): (center_y - int(center_area.shape[1] / 2) + center_area.shape[1])] = center_area
            center_area_rate = np.sum(cv.bitwise_and(vessel_mask, vessel_mask, mask=center_area_image)) / np.sum(center_area_image)
            surround_area_image = nodule_mask - center_area_image
            surround_area_rate = np.sum(cv.bitwise_and(vessel_mask, vessel_mask, mask=surround_area_image)) / np.sum(surround_area_image)
            if surround_area_rate == 0.0:
                vdr = 1000
                svp = 1
            else:
                vdr = center_area_rate / surround_area_rate
                svp = 1 if vdr >= 1.0 else 0
            # console.print('vessel density rate (VDR):{}'.format(vdr), style='yellow')
            # console.print('svp (SVP):{}'.format(svp), style='yellow')
            #TODO:计算血管直径Diameter D
            dis = ndimage.distance_transform_edt(vessel_mask / 255)
            # print(np.max(dis))
            # all_branches = leaf_obj.append(other_obj)
            diameters = []
            for i, obj in enumerate(segment_objects): 
                branch = np.zeros_like(vessel_mask)
                branch = cv.drawContours(branch, segment_objects, i, 255, 1)
                num = len(segment_objects)
                # sum_d = np.sum(cv.bitwise_and(dis, dis, mask=branch))
                # diameters.append((sum_d / num)*2)
                diameters.append(FindingTheSutiableDiameter(dis, branch, vessel_mask, segment_objects, i))
            # console.print('all vessel branches diameter (D):{}'.format(diameters), style='yellow')

            #test
            branch = np.zeros_like(vessel_mask)
            for i, obj in enumerate(segment_objects):
                branch = cv.drawContours(branch, segment_objects, i, 255, int(diameters[i]))

            #TODO:计算默里偏差MD
            md_list = []
            branch_relation = []
            for i in range(nonzero_coords.shape[0]):
                branch_list = []
                for j, obj in enumerate(segment_objects):
                    for jj in range(obj.shape[0]):
                        diss = math.sqrt((nonzero_coords[i][1] - obj[jj][0][0]) ** 2 + (nonzero_coords[i][0] - obj[jj][0][1]) ** 2)
                        if diss < 5:
                            branch_list.append(j)
                            break
                branch_dias = [diameters[j] for j in branch_list]
                branch_dias = sorted(branch_dias, reverse=True)
                # print(branch_dias)
                # if len(branch_dias) < 2:
                #     continue
                mother_branch = branch_dias[0]
                sum_son = 0.0
                for ii in range(1, len(branch_dias)):
                    sum_son += branch_dias[ii] ** 3
                md = abs(mother_branch ** 3 - sum_son) / (mother_branch ** 3)
                md_list.append(md)
                branch_relation.append(branch_list)
            # console.print('all vessel branches points mds (MD):{}'.format(md_list), style='yellow')
                



            #TODO:计算mvFP

            mvFP = CalmvFP(vessel_mask=vessel_mask)
            # console.print('microvessel Fractal Dimension (mvFP):{}'.format(mvFP), style='yellow')



            # #TODO:计算血管弯曲度Tortuosity

                        
            tortuosity_list = []
            tips_list = []
            for i, obj in enumerate(segment_objects):
                # print(obj)
                first_point = np.array([obj[0]])
                obj = np.concatenate((obj, first_point), axis=0)
                one_branch_skeleton = np.zeros_like(vessel_mask)
                one_branch_skeleton = cv.drawContours(one_branch_skeleton, segment_objects, i, 255, 1)
                # s_closed = cv.isContourConvex(obj)
                path_length = cv.arcLength(obj, closed=False) / 2
                # print(path_length)
                tips = FindTips(obj)
                tips_list.append(tips)
                # print(tips)
                if len(tips) != 2:
                    print('error!')
                
                eulcidean_length = math.sqrt((tips[0][0] - tips[1][0]) ** 2 + (tips[0][1] - tips[1][1]) ** 2)
                if eulcidean_length == 0.0:
                    # print('t : {}'.format(1.0))
                    tortuosity_list.append(1.0)
                else:
                    # print('t : {}'.format(path_length / eulcidean_length))
                    
                    tortuosity_list.append(path_length / eulcidean_length if path_length / eulcidean_length >= 1.0 else 1.0)


            # console.print('all vessel branches Tortuosity (T):{}'.format(tortuosity_list), style='yellow')
            # print(max(tortuosity_list))
            #TODO:计算分叉角度BA

            ba_list = []
            for i in range(nonzero_coords.shape[0]):
                relation_list = branch_relation[i]
                if len(relation_list) != 3:
                    continue
                max_dia = 0
                for j in relation_list:
                    if diameters[j] > max_dia:
                        father_dia_index = j
                relation_list.remove(father_dia_index)
                first_son_index = relation_list[0]
                second_son_index = relation_list[1]
                first_obj = segment_objects[first_son_index]
                second_obj = segment_objects[second_son_index]
                first_branch = cv.drawContours(np.zeros_like(vessel_mask), [first_obj], 0, 255, 1)
                non_zero_points = np.argwhere(first_branch > 0)
                distances = [np.linalg.norm(np.array(nonzero_coords[i]) - np.array(p)) for p in non_zero_points]
                nearest_indices = np.argsort(distances)[:10] if len(non_zero_points) > 10 else np.argsort(distances)[:len(non_zero_points)]
                first_nearest_points = [non_zero_points[i] for i in nearest_indices]
                second_branch = cv.drawContours(np.zeros_like(vessel_mask), [second_obj], 0, 255, 1)
                non_zero_points = np.argwhere(second_branch > 0)
                distances = [np.linalg.norm(np.array(nonzero_coords[i]) - np.array(p)) for p in non_zero_points]
                nearest_indices = np.argsort(distances)[:10] if len(non_zero_points) > 10 else np.argsort(distances)[:len(non_zero_points)]
                second_nearest_points = [non_zero_points[i] for i in nearest_indices]
                v1 = np.array([first_nearest_points[-1][0] - nonzero_coords[i][0], first_nearest_points[-1][1] - nonzero_coords[i][1]])
                v2 = np.array([second_nearest_points[-1][0] - nonzero_coords[i][0], second_nearest_points[-1][1] - nonzero_coords[i][1]])
                cosine_angle = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
                if cosine_angle > 1.0:
                    cosine_angle = 1.0
                elif cosine_angle < -1.0:
                    cosine_angle = -1.0
                # print(cosine_angle)
                angle_in_radians = np.arccos(cosine_angle)
                angle_in_degrees = np.degrees(angle_in_radians)
                ba_list.append(angle_in_degrees)
                #tesst
                if angle_in_degrees == 0.0:
                    show_img = np.zeros_like(vessel_mask)
                    # show_img = cv.drawContours(show_img, segment_objects, first_son_index, 255, 1)
                    show_img = cv.drawContours(show_img, segment_objects, second_son_index, 255, 1)
                    # show_img[nonzero_coords[i][0], nonzero_coords[i][1]] = 255
                    # show_img[first_nearest_points[-1][0], first_nearest_points[-1][1]] = 255
                    # show_img[second_nearest_points[-1][0], second_nearest_points[-1][1]] = 255
                    cv.imshow('show', show_img)
                    cv.waitKey(0)
                # print(index_list)
                # pcv.morphology.segment_insertion_angle
            # console.print('all vessel branches BA (BA)):{}'.format(ba_list), style='yellow')

            # print(segment_insertion_angles)
            features['name'].append(file[:-4])
            features['nb'].append(nb)
            features['nv'].append(nv)
            features['vd'].append(vd)
            features['vdr'].append(vdr)
            features['svp'].append(svp)
            
            features['diameters'].append(diameters)
            features['ave_diameter'].append(calculate_average(diameters))
            features['max_diameter'].append(max(diameters))
            features['min_diameter'].append(min(diameters))
            if len(md_list) != 0:
                features['md_list'].append(md_list)
                features['ave_md'].append(calculate_average(md_list))
                features['min_md'].append(min(md_list))
                features['max_md'].append(max(md_list))
            else:
                features['md_list'].append(md_list)
                features['ave_md'].append(10000)
                features['min_md'].append(10000)
                features['max_md'].append(10000)
            features['mvFP'].append(mvFP)
            features['tortuosity_list'].append(tortuosity_list)
            features['ave_tortuosity'].append(calculate_average(tortuosity_list))
            features['min_tortuosity'].append(min(tortuosity_list))
            features['max_tortuosity'].append(max(tortuosity_list))
            if len(ba_list) != 0:
                features['ba_list'].append(ba_list)
                features['ave_ba'].append(calculate_average(ba_list))
                features['min_ba'].append(min(ba_list))
                features['max_ba'].append(max(ba_list))
            else:
                features['ba_list'].append(ba_list)
                features['ave_ba'].append(0)
                features['min_ba'].append(0)
                features['max_ba'].append(0)
            dia = max(index[1] - index[0], index[3] - index[2])
            # dia = (float(rod) / 550) * dia
            features['noddule_dia'].append(dia)
            features['area'].append((np.sum(nodule_mask) / 255))
            if dir_path[-2:] == 'gn' or dir_path[-2:] == 'n1':
                features['class'].append(0)
            else:
                features['class'].append(1)
del features['md_list']
del features['tortuosity_list']
del features['ba_list']
del features['diameters']
        
features_df = pd.DataFrame(features)
features_df.to_excel('all_features.xlsx', index=False)


        # cv.imshow('rectangle_roi', rectangle_roi)

        # cv.imshow('vessel_area', vessel_image)
        # # branch = cv.bitwise_and(branch, branch, mask=pruned_skeleton)
        # cv.imshow('segment_objects', pruned_skeleton_show_image)

        # cv.waitKey(2000)
        # cv.destroyAllWindows()
        # plt.imshow(dis)
        # plt.show()